In [ ]:
from pymatgen.core import Structure
from smol.cofe import ClusterSubspace
from smol.moca import ClusterExpansionProcessor
import numpy as np

# 创建 subspace
unit = Structure.from_file('../cluster_expansion_code/POSCAR.POSCAR.vasp')
prim = unit.copy()
prim.replace_species({
    'Fe': {'Cu': 1/7, 'Ni': 1/7, 'Fe': 1/7, 'Al': 1/7, 'Cr': 1/7, 'Co': 1/7, 'Li': 1/7},
    'O': {'O': 1/2, 'F': 1/2}
})

subspace = ClusterSubspace.from_cutoffs(
    prim, ltol=0.2, stol=0.3, angle_tol=5,
    cutoffs={2: 6.0, 3: 4.0}, basis='indicator', supercell_size='volume'
)

sc_matrix = np.array([[3, 0, 0], [0, 3, 0], [0, 0, 3]])

# 加载 npz 结构
data1 = np.load('local_structures/step0_vac13457_X.npz', allow_pickle=True)
template1 = Structure(data1['cell'], data1['symbols'], data1['positions'], coords_are_cartesian=True)

data2 = np.load('local_structures/step70000_vac354749_XO.npz', allow_pickle=True)
template2 = Structure(data2['cell'], data2['symbols'], data2['positions'], coords_are_cartesian=True)

In [ ]:
# 看看 correct 的结果
dummy_coefficients = np.random.rand(subspace.num_corr_functions)
processor = ClusterExpansionProcessor(subspace, sc_matrix, dummy_coefficients)
occu1_struc = processor.occupancy_from_structure(template1)
print('Correct occupancy 1 (first 20):', occu1_struc[:20])

occu2_struc = processor.occupancy_from_structure(template2)
print('Correct occupancy 2 (first 20):', occu2_struc[:20])

Correct occupancy 1 (first 20): [0 0 0 4 5 2 0 0 0 0 0 0 3 1 4 0 0 0 0 0]
Correct occupancy 2 (first 20): [0 0 0 0 0 0 0 0 0 3 0 1 2 1 3 0 0 1 3 0]
Correct occupancy 2 (first 20): [0 0 0 0 0 0 0 0 0 3 0 1 2 1 3 0 0 1 3 0]


In [ ]:
# 创建超胞结构用于 site_mapping
supercell = subspace.structure.copy()
_ = supercell.make_supercell(sc_matrix)

In [ ]:
# 获取 site_mapping
site_mapping1 = subspace.structure_site_mapping(supercell, template1)
site_mapping2 = subspace.structure_site_mapping(supercell, template2)
print(f"Site mapping 1 (前20): {site_mapping1[:20]}")
print(f"Site mapping 2 (前20): {site_mapping2[:20]}")

Site mapping 1 (前20): [54, 56, 55, 170, 169, 168, 60, 62, 61, 167, 166, 165, 57, 59, 58, 164, 163, 162, 947, 946]
Site mapping 2 (前20): [54, 56, 55, 170, 169, 168, 60, 62, 61, 167, 166, 165, 57, 59, 58, 164, 163, 162, 947, 946]


In [ ]:
assert site_mapping1 == site_mapping2

In [ ]:
site_mapping1.index(1)

128

In [ ]:
site_mapping1.index(54)

0

In [ ]:
occu1 = []
for i, allowed_species in enumerate(processor.allowed_species):    

    spec = template1[site_mapping1.index(i)].specie
    occu1.append(allowed_species.index(spec))
occu1 = np.array(occu1)

occu2 = []
for i, allowed_species in enumerate(processor.allowed_species):    
    
    spec = template2[site_mapping2.index(i)].specie
    occu2.append(allowed_species.index(spec))
occu2 = np.array(occu2)


In [ ]:
assert np.allclose(occu1, occu1_struc)
assert np.allclose(occu2, occu2_struc)